In [1]:
import pandas as pd
import pickle

In [2]:
with open("../db/df.pkl", 'rb') as f:
    df = pickle.load(f)
with open('../model/rndf.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
df.columns

Index(['Price', 'Area', 'No. of Bedrooms', 'Resale', 'MaintenanceStaff',
       'Gymnasium', 'SwimmingPool', 'LandscapedGardens', 'JoggingTrack',
       'RainWaterHarvesting', 'IndoorGames', 'ShoppingMall', 'Intercom',
       'SportsFacility', 'ATM', 'ClubHouse', 'School', '24X7Security',
       'PowerBackup', 'CarParking', 'StaffQuarter', 'Cafeteria',
       'MultipurposeRoom', 'Hospital', 'WashingMachine', 'Gasconnection', 'AC',
       'Wifi', 'Children'splayarea', 'LiftAvailable', 'BED', 'VaastuCompliant',
       'Microwave', 'GolfCourse', 'TV', 'DiningTable', 'Sofa', 'Wardrobe',
       'Stadium', 'Town_Bangalore', 'Town_Chennai', 'Town_Delhi',
       'Town_Hyderabad', 'Town_Kolkata', 'Town_Mumbai'],
      dtype='object')

In [50]:
def balls(price):
    
    try:
        df_element_higher = df[df['Price']> price].sample(n=1)
    except:
        df_element_higher = pd.DataFrame()
        
    try:
        df_element_lower = df[df['Price']< price].sample(n=1)
    except:
        df_element_lower = pd.DataFrame()
        
    try:
        df_element_same = df[df['Price'] == price].sample(n=1)
    except:
        df_element_same = pd.DataFrame()
    
    return [df_element_higher,df_element_lower,df_element_same]

In [43]:
a = balls(19999999)
a[2].head()

""


In [ ]:
import tkinter as tk 
NUMBERS_OF_ROOMS = df["No. of Bedrooms"].unique()
ALL_AREAS = df['Area'].unique()
columns_of_df = df.columns[3:-6]
root = tk.Tk()

def get_columns():
    return columns_of_df


OPTIONS = [
"Bangalore",
"Chennai",
"Delhi",
"Hyderabad",
"Kolkata",
"Mumbai"
]

rooms_n = tk.StringVar(root)
rooms_n.set(NUMBERS_OF_ROOMS[0]) # default value

area_n = tk.StringVar(root)
area_n.set(ALL_AREAS[0]) # default value

variable = tk.StringVar(root)
variable.set(OPTIONS[0]) # default value

frame = tk.Frame(root)
frame.grid(row = 0, column = 0)

def my_function():
    columns_of_df = get_columns()
    
    
    answers = list()
    
    room = rooms_n.get()
    answers.append(room)
    
    area = area_n.get()
    answers.append(area)
    for i in columns_of_df:
        try:
            num = int(enstries[f"{i}_ent"].get())
            if num > 1:
                answers.append(1)
            else:
                answers.append(num)
            
        except:
            answers.append(0)
            
    
    columns_of_df = columns_of_df.insert(0,"No. of Bedrooms")
    columns_of_df = columns_of_df.insert(0,"Area")
    values = pd.DataFrame([answers],columns = columns_of_df)
    
    towns = pd.DataFrame([[0,0,0,0,0,0]],columns=['Town_Bangalore', 'Town_Chennai', 'Town_Delhi',
       'Town_Hyderabad', 'Town_Kolkata', 'Town_Mumbai'])
    towns_columns = towns.columns
    town = variable.get()
    town_col = [col for col in towns.columns if f'{town}' in col]
    towns[town_col]=1
    
    df_merged = pd.concat([values, towns],axis=1, join="inner")
    print(answers)
    print(towns.head())
    print(len(answers))
    print(df_merged.head())
    
    for widget in frame.winfo_children():
        widget.destroy()
        # this will clear frame and frame will be empty
        # if you want to hide the empty panel then
        frame.grid_forget()
    price = model.predict(df_merged)
    
    df_elements = balls(int(price))
    
    room_label = tk.Label(root, text = f"Цена вашей квартиры {price}")
    room_label.grid(row = 0, column = 0)
    room_label_1 = tk.Label(root, text = f"")
    room_label_1.grid(row = 0, column = 1)
    
    room_label_higher = tk.Label(root, text = f"Рекомендованая квартира за цену выше")
    room_label_higher.grid(row = 1, column = 0)
    room_label_higher_df = tk.Label(root, text = f"{df_elements[0].head()}")
    room_label_higher_df.grid(row = 1, column = 1)
    
    room_label_same = tk.Label(root, text = f"Рекомендованая квартира за цену ниже")
    room_label_same.grid(row = 2, column = 0)
    room_label_same_df = tk.Label(root, text = f"{df_elements[1].head()}")
    room_label_same_df.grid(row = 2, column = 1)
    
    room_label_lower = tk.Label(root, text = f"Рекомендованая квартира за такую-же цену")
    room_label_lower.grid(row = 3, column = 0)
    room_label_lower_df = tk.Label(root, text = f"{df_elements[2].head()}")
    room_label_lower_df.grid(row = 3, column = 1)
    
    #do stuff with url_member
rooms_labels = tk.Label(root, text = "Rooms ")
rooms_labels.grid(row = 0, column = 0)
rooms_ent = tk.OptionMenu(root, rooms_n, *NUMBERS_OF_ROOMS)
rooms_ent.grid(row = 0, column = 1)

area_labels = tk.Label(root, text = "Areas ")
area_labels.grid(row = 1, column = 0)
area_ent = tk.OptionMenu(root, area_n, *ALL_AREAS)
area_ent.grid(row = 1, column = 1)

row_n = 2
enstries={}
for i in columns_of_df:
    enstries[i] = tk.Label(root, text = f"{i}")
    enstries[i].grid(row = row_n, column = 0)
    enstries[f"{i}_ent"] = tk.Entry(root)
    enstries[f"{i}_ent"].grid(row = row_n, column = 1)

    row_n +=1
town_labels = tk.Label(root, text = "towns ")
town_labels.grid(row = row_n, column = 0)
town_ent = tk.OptionMenu(root, variable, *OPTIONS)
town_ent.grid(row = row_n, column = 1)

my_button = tk.Button(root, text = "Submit", command = my_function)
my_button.grid(row = row_n+2, column = 1)

print(enstries)

root.mainloop()

{'Resale': <tkinter.Label object .!label3>, 'Resale_ent': <tkinter.Entry object .!entry>, 'MaintenanceStaff': <tkinter.Label object .!label4>, 'MaintenanceStaff_ent': <tkinter.Entry object .!entry2>, 'Gymnasium': <tkinter.Label object .!label5>, 'Gymnasium_ent': <tkinter.Entry object .!entry3>, 'SwimmingPool': <tkinter.Label object .!label6>, 'SwimmingPool_ent': <tkinter.Entry object .!entry4>, 'LandscapedGardens': <tkinter.Label object .!label7>, 'LandscapedGardens_ent': <tkinter.Entry object .!entry5>, 'JoggingTrack': <tkinter.Label object .!label8>, 'JoggingTrack_ent': <tkinter.Entry object .!entry6>, 'RainWaterHarvesting': <tkinter.Label object .!label9>, 'RainWaterHarvesting_ent': <tkinter.Entry object .!entry7>, 'IndoorGames': <tkinter.Label object .!label10>, 'IndoorGames_ent': <tkinter.Entry object .!entry8>, 'ShoppingMall': <tkinter.Label object .!label11>, 'ShoppingMall_ent': <tkinter.Entry object .!entry9>, 'Intercom': <tkinter.Label object .!label12>, 'Intercom_ent': <tkint

C:\Users\Strange\AppData\Local\Temp\ipykernel_9448\481060439.py:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  df_elements = balls(int(price))
